### Deliverable 1: Preprocessing the Data for a Neural Network

In [98]:
# Import our dependencies
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf
import os
from tensorflow.keras.callbacks import ModelCheckpoint

os.makedirs('checkpoints/', exist_ok=True)
checkpoint_path = 'chcekpoints/weights.{epoch:02d}'

In [99]:
app_df = pd.read_csv("charity_data.csv")


In [100]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
app_df = app_df.drop(columns=['EIN','NAME']).copy()

In [101]:
# Determine the number of unique values in each column.
categoricals = app_df.columns[app_df.dtypes == 'object']

In [102]:
# Determine which values to replace if counts are less than ...?
replace_application = ['T13', 'T12', 'T2', 'T25', 'T14', 'T29', 'T15', 'T17']
# Replace in dataframe
for app in replace_application:
    app_df.APPLICATION_TYPE = app_df.APPLICATION_TYPE.replace(app,"Other")
    
# Check to make sure binning was successful


In [103]:
# Look at CLASSIFICATION value counts for binning
class_counts = app_df.CLASSIFICATION.value_counts()


In [104]:
# Determine which values to replace if counts are less than ..?
replace_class = []
row = class_counts.index
for i in range (0,len(class_counts)):
    if class_counts[i] < 100:
        replace_class.append(row[i])
        
        # Replace in dataframe
for cls in replace_class:
    app_df.CLASSIFICATION = app_df.CLASSIFICATION.replace(cls,"Other")
    
# Check to make sure binning was successful


In [105]:
# Generate our categorical variable lists
categorical_columns = app_df.columns[app_df.dtypes == 'object']

In [106]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(app_df[categorical_columns]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(categorical_columns)


In [107]:
# Merge one-hot encoded features and drop the originals
app_df = app_df.merge(encode_df, left_index=True, right_index=True)


In [108]:
app_df.drop(columns=categorical_columns,inplace=True)


In [109]:
# Split our preprocessed data into our features and target arrays
y_raw = app_df['IS_SUCCESSFUL'].values.reshape(-1,1)
X_raw = app_df.drop(columns='IS_SUCCESSFUL')

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X_raw, y_raw, random_state=42)

In [110]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [111]:
X_train_scaled.shape

(25724, 50)

### Deliverable 3: Optimize the model

#### Create a function to create and run neutal network models

In [112]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
def make_run_evaluate_nn_model(X_train_data, X_test_data, y_train_data, y_test_data, layer_1_units, 
                               activation_1, activation_2, make_layer_2='yes', layer_2_units=5, num_epochs='50'):
    
    nn = tf.keras.models.Sequential()

    # First hidden layer
    nn.add(tf.keras.layers.Dense(units=layer_1_units, activation = activation_1, input_dim = 50))

    # Second hidden layer
    if make_layer_2 == 'yes':
        nn.add(tf.keras.layers.Dense(units=layer_2_units, activation = activation_2))

    # Output layer
    nn.add(tf.keras.layers.Dense(units = 1, activation = 'sigmoid'))

    # Check the structure of the model
    nn.summary()

    nn.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

    fit_model = nn.fit(X_train_data, y_train_data, epochs=50)

    model_loss, model_accuracy = nn.evaluate(X_test_data,y_test_data,verbose=2)
    print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")
    
    
    return [model_accuracy, layer_1_units, layer_2_units]

In [113]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
def make_run_evaluate_nn_model_with_callbacks(X_train_data, X_test_data, y_train_data, y_test_data, layer_1_units, 
                               activation_1, activation_2, make_layer_2='yes', layer_2_units=5, num_epochs='50'):
    
    nn = tf.keras.models.Sequential()

    # First hidden layer
    nn.add(tf.keras.layers.Dense(units=layer_1_units, activation = activation_1, input_dim = 50))

    # Second hidden layer
    if make_layer_2 == 'yes':
        nn.add(tf.keras.layers.Dense(units=layer_2_units, activation = activation_2))

    # Output layer
    nn.add(tf.keras.layers.Dense(units = 1, activation = 'sigmoid'))

    # Check the structure of the model
    nn.summary()

    nn.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
    
    cp_callback = ModelCheckpoint(
        filepath = checkpoint_path,
        verbose=1,
        save_weights_only=True,
        save_freq = 'epoch')

    fit_model = nn.fit(X_train_data, y_train_data, epochs=50, callbacks=[cp_callback])

    model_loss, model_accuracy = nn.evaluate(X_test_data,y_test_data,verbose=2)
    print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")
    
    nn.save('AlphabetSoupCharity_Optimization.h5')
    


### First attempt to optimize the model.  Loop to test a single layer model with up to 10 layers

In [114]:
# Single hidden layer with from 1 to 10 neurons
results = []
for index in range (50,60):
    results.append(make_run_evaluate_nn_model(X_train_scaled, X_test_scaled, y_train, y_test, index, 
                               'relu', 'no', 50))

Model: "sequential_68"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_204 (Dense)            (None, 50)                2550      
_________________________________________________________________
dense_205 (Dense)            (None, 1)                 51        
Total params: 2,601
Trainable params: 2,601
Non-trainable params: 0
_________________________________________________________________
Epoch 1/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5780 - accuracy: 0.7172
Epoch 2/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5585 - accuracy: 0.7272
Epoch 3/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5545 - accuracy: 0.7296
Epoch 4/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5528 - accuracy: 0.7286
Epoch 5/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5508 - accuracy: 0.7297
Epoch 6/50
80

804/804 [==============================] - 1s 1ms/step - loss: 0.5438 - accuracy: 0.7315
Epoch 20/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5435 - accuracy: 0.7321
Epoch 21/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5440 - accuracy: 0.7313
Epoch 22/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5433 - accuracy: 0.7325
Epoch 23/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5434 - accuracy: 0.7318
Epoch 24/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5428 - accuracy: 0.7318
Epoch 25/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5425 - accuracy: 0.7330
Epoch 26/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5428 - accuracy: 0.7329
Epoch 27/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5420 - accuracy: 0.7327
Epoch 28/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5427 - accuracy: 0.7336
Ep

804/804 [==============================] - 1s 1ms/step - loss: 0.5412 - accuracy: 0.7339
Epoch 44/50
804/804 [==============================] - 1s 999us/step - loss: 0.5412 - accuracy: 0.7345
Epoch 45/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5407 - accuracy: 0.7347
Epoch 46/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5410 - accuracy: 0.7342
Epoch 47/50
804/804 [==============================] - 1s 996us/step - loss: 0.5409 - accuracy: 0.7344
Epoch 48/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5400 - accuracy: 0.7350
Epoch 49/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5403 - accuracy: 0.7363
Epoch 50/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5400 - accuracy: 0.7346
268/268 - 0s - loss: 0.5524 - accuracy: 0.7238
Loss: 0.552400529384613, Accuracy: 0.7238484025001526
Model: "sequential_71"
_________________________________________________________________
Layer (typ

804/804 [==============================] - 1s 1ms/step - loss: 0.5461 - accuracy: 0.7315
Epoch 11/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5463 - accuracy: 0.7307
Epoch 12/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5455 - accuracy: 0.7331
Epoch 13/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5450 - accuracy: 0.7315
Epoch 14/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5447 - accuracy: 0.7332
Epoch 15/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5448 - accuracy: 0.7324
Epoch 16/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5446 - accuracy: 0.7314
Epoch 17/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5444 - accuracy: 0.7331
Epoch 18/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5438 - accuracy: 0.7332
Epoch 19/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5441 - accuracy: 0.7320
Ep

804/804 [==============================] - 1s 1ms/step - loss: 0.5423 - accuracy: 0.7330
Epoch 35/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5417 - accuracy: 0.7339
Epoch 36/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5411 - accuracy: 0.7343
Epoch 37/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5415 - accuracy: 0.7338
Epoch 38/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5412 - accuracy: 0.7347
Epoch 39/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5415 - accuracy: 0.7337
Epoch 40/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5411 - accuracy: 0.7338
Epoch 41/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5414 - accuracy: 0.7339
Epoch 42/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5415 - accuracy: 0.7339
Epoch 43/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5409 - accuracy: 0.7338
Ep

804/804 [==============================] - 1s 1ms/step - loss: 0.5814 - accuracy: 0.7123
Epoch 2/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5583 - accuracy: 0.7280
Epoch 3/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5537 - accuracy: 0.7294
Epoch 4/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5513 - accuracy: 0.7293
Epoch 5/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5506 - accuracy: 0.7281
Epoch 6/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5488 - accuracy: 0.7300
Epoch 7/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5488 - accuracy: 0.7290
Epoch 8/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5481 - accuracy: 0.7304
Epoch 9/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5476 - accuracy: 0.7298
Epoch 10/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5469 - accuracy: 0.7315
Epoch 11/5

804/804 [==============================] - 1s 1ms/step - loss: 0.5431 - accuracy: 0.7320
Epoch 26/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5429 - accuracy: 0.7314
Epoch 27/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5429 - accuracy: 0.7331
Epoch 28/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5429 - accuracy: 0.7342
Epoch 29/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5429 - accuracy: 0.7336
Epoch 30/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5418 - accuracy: 0.7335
Epoch 31/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5428 - accuracy: 0.7339
Epoch 32/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5424 - accuracy: 0.7329
Epoch 33/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5423 - accuracy: 0.7326
Epoch 34/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5421 - accuracy: 0.7337
Ep

804/804 [==============================] - 1s 1ms/step - loss: 0.5399 - accuracy: 0.7352
Epoch 50/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5401 - accuracy: 0.7342
268/268 - 0s - loss: 0.5526 - accuracy: 0.7278
Loss: 0.5525969862937927, Accuracy: 0.7278134226799011


In [115]:
results

[[0.7287463545799255, 50, 5],
 [0.7302623987197876, 51, 5],
 [0.7238484025001526, 52, 5],
 [0.7269970774650574, 53, 5],
 [0.732478141784668, 54, 5],
 [0.7276967763900757, 55, 5],
 [0.728396475315094, 56, 5],
 [0.7321282625198364, 57, 5],
 [0.727580189704895, 58, 5],
 [0.7278134226799011, 59, 5]]

### Second attempt to optimize the model.
### Nested for loops to test a first hidden layer with from 1 to 10 neurons and a second hiden layer with from 1 up to the number of neurons in the first layer

In [116]:
results = []
for first in range (50,60):
    for second in range (25, 30):
        results.append(make_run_evaluate_nn_model(X_train_scaled, X_test_scaled, y_train, y_test, first, 
                               'relu', 'relu','yes', second, 50))

Model: "sequential_78"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_224 (Dense)            (None, 50)                2550      
_________________________________________________________________
dense_225 (Dense)            (None, 25)                1275      
_________________________________________________________________
dense_226 (Dense)            (None, 1)                 26        
Total params: 3,851
Trainable params: 3,851
Non-trainable params: 0
_________________________________________________________________
Epoch 1/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5730 - accuracy: 0.7163
Epoch 2/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5540 - accuracy: 0.7287
Epoch 3/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5502 - accuracy: 0.7292
Epoch 4/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5486 

804/804 [==============================] - 1s 1ms/step - loss: 0.5412 - accuracy: 0.7357
Epoch 17/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5415 - accuracy: 0.7335
Epoch 18/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5404 - accuracy: 0.7342
Epoch 19/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5407 - accuracy: 0.7341
Epoch 20/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5398 - accuracy: 0.7357
Epoch 21/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5400 - accuracy: 0.7344
Epoch 22/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5396 - accuracy: 0.7342
Epoch 23/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5394 - accuracy: 0.7364
Epoch 24/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5391 - accuracy: 0.7357
Epoch 25/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5391 - accuracy: 0.7348
Ep

804/804 [==============================] - 1s 1ms/step - loss: 0.5362 - accuracy: 0.7383
Epoch 39/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5366 - accuracy: 0.7365
Epoch 40/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5362 - accuracy: 0.7378
Epoch 41/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5366 - accuracy: 0.7373
Epoch 42/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5364 - accuracy: 0.7380
Epoch 43/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5358 - accuracy: 0.7389
Epoch 44/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5362 - accuracy: 0.7384
Epoch 45/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5361 - accuracy: 0.7389
Epoch 46/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5359 - accuracy: 0.7388
Epoch 47/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5354 - accuracy: 0.7399
Ep

804/804 [==============================] - 1s 1ms/step - loss: 0.5516 - accuracy: 0.7288
Epoch 3/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5494 - accuracy: 0.7296
Epoch 4/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5476 - accuracy: 0.7295
Epoch 5/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5467 - accuracy: 0.7321
Epoch 6/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5456 - accuracy: 0.7322
Epoch 7/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5444 - accuracy: 0.7329
Epoch 8/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5441 - accuracy: 0.7337
Epoch 9/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5434 - accuracy: 0.7327
Epoch 10/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5430 - accuracy: 0.7354
Epoch 11/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5425 - accuracy: 0.7344
Epoch 12/

804/804 [==============================] - 1s 1ms/step - loss: 0.5388 - accuracy: 0.7355
Epoch 25/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5388 - accuracy: 0.7365
Epoch 26/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5385 - accuracy: 0.7357
Epoch 27/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5386 - accuracy: 0.7362
Epoch 28/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5382 - accuracy: 0.7360
Epoch 29/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5381 - accuracy: 0.7372
Epoch 30/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5379 - accuracy: 0.7364
Epoch 31/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5376 - accuracy: 0.7385
Epoch 32/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5374 - accuracy: 0.7366
Epoch 33/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5374 - accuracy: 0.7367
Ep

...

804/804 [==============================] - 1s 1ms/step - loss: 0.5442 - accuracy: 0.7333
Epoch 10/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5436 - accuracy: 0.7341
Epoch 11/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5425 - accuracy: 0.7341
Epoch 12/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5427 - accuracy: 0.7334
Epoch 13/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5421 - accuracy: 0.7333
Epoch 14/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5422 - accuracy: 0.7334
Epoch 15/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5409 - accuracy: 0.7364
Epoch 16/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5413 - accuracy: 0.7338
Epoch 17/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5406 - accuracy: 0.7357
Epoch 18/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5403 - accuracy: 0.7357
Ep

804/804 [==============================] - 1s 1ms/step - loss: 0.5380 - accuracy: 0.7369
Epoch 32/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5374 - accuracy: 0.7370
Epoch 33/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5374 - accuracy: 0.7372
Epoch 34/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5369 - accuracy: 0.7376
Epoch 35/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5375 - accuracy: 0.7370
Epoch 36/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5372 - accuracy: 0.7373
Epoch 37/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5371 - accuracy: 0.7378
Epoch 38/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5372 - accuracy: 0.7368
Epoch 39/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5364 - accuracy: 0.7371
Epoch 40/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5368 - accuracy: 0.7372
Ep

804/804 [==============================] - 1s 1ms/step - loss: 0.5732 - accuracy: 0.7135
Epoch 2/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5535 - accuracy: 0.7288
Epoch 3/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5501 - accuracy: 0.7304
Epoch 4/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5486 - accuracy: 0.7306
Epoch 5/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5471 - accuracy: 0.7316
Epoch 6/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5460 - accuracy: 0.7322
Epoch 7/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5455 - accuracy: 0.7324
Epoch 8/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5451 - accuracy: 0.7326
Epoch 9/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5447 - accuracy: 0.7332
Epoch 10/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5436 - accuracy: 0.7341
Epoch 11/5

804/804 [==============================] - 1s 1ms/step - loss: 0.5391 - accuracy: 0.7372
Epoch 24/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5391 - accuracy: 0.7369
Epoch 25/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5388 - accuracy: 0.7369
Epoch 26/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5384 - accuracy: 0.7369
Epoch 27/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5381 - accuracy: 0.7364
Epoch 28/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5382 - accuracy: 0.7370
Epoch 29/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5378 - accuracy: 0.7367
Epoch 30/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5376 - accuracy: 0.7372
Epoch 31/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5379 - accuracy: 0.7368
Epoch 32/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5372 - accuracy: 0.7376
Ep

804/804 [==============================] - 1s 1ms/step - loss: 0.5354 - accuracy: 0.7383
Epoch 46/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5354 - accuracy: 0.7376
Epoch 47/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5352 - accuracy: 0.7389
Epoch 48/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5352 - accuracy: 0.7385
Epoch 49/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5350 - accuracy: 0.7381
Epoch 50/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5349 - accuracy: 0.7384
268/268 - 0s - loss: 0.5536 - accuracy: 0.7312
Loss: 0.5535534620285034, Accuracy: 0.731195330619812
Model: "sequential_127"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_371 (Dense)            (None, 59)                3009      
_________________________________________________________________
dense_372 (Dense)

In [117]:
results.sort()

In [118]:
results

[[0.7230320572853088, 50, 26],
 [0.7230320572853088, 52, 25],
 [0.7253644466400146, 51, 27],
 [0.7267638444900513, 50, 25],
 [0.7269970774650574, 55, 27],
 [0.7271137237548828, 58, 28],
 [0.7273469567298889, 53, 28],
 [0.7276967763900757, 57, 27],
 [0.7278134226799011, 58, 27],
 [0.7281632423400879, 57, 28],
 [0.7282798886299133, 53, 25],
 [0.7282798886299133, 54, 27],
 [0.7282798886299133, 54, 29],
 [0.7282798886299133, 55, 25],
 [0.7282798886299133, 55, 26],
 [0.7282798886299133, 56, 26],
 [0.728396475315094, 59, 29],
 [0.7286297082901001, 56, 29],
 [0.728863000869751, 56, 28],
 [0.7289795875549316, 53, 27],
 [0.7290962338447571, 57, 25],
 [0.7292128205299377, 50, 28],
 [0.7292128205299377, 55, 28],
 [0.7292128205299377, 57, 26],
 [0.7293294668197632, 52, 29],
 [0.7294460535049438, 51, 25],
 [0.7295626997947693, 51, 28],
 [0.7295626997947693, 58, 25],
 [0.72967928647995, 52, 28],
 [0.72967928647995, 53, 26],
 [0.729912519454956, 50, 27],
 [0.7300291657447815, 54, 26],
 [0.73002916574

### Third attempt to optimize the model.
### Same loop as the second attempt but with elu activation.

In [119]:
results = []
for first in range (50, 60):
    for second in range (25, 30):
        results.append(make_run_evaluate_nn_model(X_train_scaled, X_test_scaled, y_train, y_test, first, 
                               'elu', 'elu','yes', second, 50))

Model: "sequential_128"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_374 (Dense)            (None, 50)                2550      
_________________________________________________________________
dense_375 (Dense)            (None, 25)                1275      
_________________________________________________________________
dense_376 (Dense)            (None, 1)                 26        
Total params: 3,851
Trainable params: 3,851
Non-trainable params: 0
_________________________________________________________________
Epoch 1/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5764 - accuracy: 0.7158
Epoch 2/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5584 - accuracy: 0.7257
Epoch 3/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5542 - accuracy: 0.7289
Epoch 4/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5524

804/804 [==============================] - 1s 1ms/step - loss: 0.5449 - accuracy: 0.7317
Epoch 17/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5446 - accuracy: 0.7311
Epoch 18/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5439 - accuracy: 0.7325
Epoch 19/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5439 - accuracy: 0.7320
Epoch 20/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5435 - accuracy: 0.7327
Epoch 21/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5438 - accuracy: 0.7330
Epoch 22/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5437 - accuracy: 0.7335
Epoch 23/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5425 - accuracy: 0.7339
Epoch 24/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5429 - accuracy: 0.7329
Epoch 25/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5421 - accuracy: 0.7341
Ep

804/804 [==============================] - 1s 1ms/step - loss: 0.5398 - accuracy: 0.7347
Epoch 39/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5393 - accuracy: 0.7347
Epoch 40/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5387 - accuracy: 0.7367
Epoch 41/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5394 - accuracy: 0.7360
Epoch 42/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5392 - accuracy: 0.7349
Epoch 43/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5387 - accuracy: 0.7357
Epoch 44/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5387 - accuracy: 0.7357
Epoch 45/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5387 - accuracy: 0.7350
Epoch 46/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5384 - accuracy: 0.7358
Epoch 47/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5385 - accuracy: 0.7367
Ep

804/804 [==============================] - 1s 1ms/step - loss: 0.5571 - accuracy: 0.7270
Epoch 3/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5534 - accuracy: 0.7281
Epoch 4/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5518 - accuracy: 0.7289
Epoch 5/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5503 - accuracy: 0.7297
Epoch 6/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5500 - accuracy: 0.7296
Epoch 7/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5483 - accuracy: 0.7297
Epoch 8/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5480 - accuracy: 0.7286
Epoch 9/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5475 - accuracy: 0.7301
Epoch 10/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5464 - accuracy: 0.7304
Epoch 11/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5466 - accuracy: 0.7304
Epoch 12/

804/804 [==============================] - 1s 1ms/step - loss: 0.5426 - accuracy: 0.7326
Epoch 25/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5424 - accuracy: 0.7335
Epoch 26/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5423 - accuracy: 0.7343
Epoch 27/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5416 - accuracy: 0.7340
Epoch 28/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5415 - accuracy: 0.7351
Epoch 29/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5414 - accuracy: 0.7354
Epoch 30/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5414 - accuracy: 0.7339
Epoch 31/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5412 - accuracy: 0.7333
Epoch 32/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5407 - accuracy: 0.7343
Epoch 33/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5405 - accuracy: 0.7355
Ep

...

804/804 [==============================] - 1s 1ms/step - loss: 0.5473 - accuracy: 0.7313
Epoch 10/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5471 - accuracy: 0.7306
Epoch 11/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5459 - accuracy: 0.7315
Epoch 12/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5458 - accuracy: 0.7299
Epoch 13/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5452 - accuracy: 0.7321
Epoch 14/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5445 - accuracy: 0.7326
Epoch 15/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5448 - accuracy: 0.7306
Epoch 16/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5437 - accuracy: 0.7336
Epoch 17/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5435 - accuracy: 0.7320
Epoch 18/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5435 - accuracy: 0.7328
Ep

804/804 [==============================] - 1s 1ms/step - loss: 0.5408 - accuracy: 0.7353
Epoch 32/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5400 - accuracy: 0.7363
Epoch 33/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5402 - accuracy: 0.7343
Epoch 34/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5402 - accuracy: 0.7362
Epoch 35/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5397 - accuracy: 0.7348
Epoch 36/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5396 - accuracy: 0.7349
Epoch 37/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5391 - accuracy: 0.7352: 0s - loss: 0.5419 - accuracy
Epoch 38/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5393 - accuracy: 0.7357
Epoch 39/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5391 - accuracy: 0.7361
Epoch 40/50
804/804 [==============================] - 1s 1ms/step - loss

804/804 [==============================] - 1s 1ms/step - loss: 0.5789 - accuracy: 0.7153
Epoch 2/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5582 - accuracy: 0.7268
Epoch 3/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5541 - accuracy: 0.7286
Epoch 4/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5535 - accuracy: 0.7282
Epoch 5/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5514 - accuracy: 0.7287
Epoch 6/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5498 - accuracy: 0.7296
Epoch 7/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5503 - accuracy: 0.7297
Epoch 8/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5484 - accuracy: 0.7311
Epoch 9/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5483 - accuracy: 0.7316
Epoch 10/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5477 - accuracy: 0.7317
Epoch 11/5

804/804 [==============================] - 1s 1ms/step - loss: 0.5421 - accuracy: 0.7340
Epoch 24/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5419 - accuracy: 0.7345
Epoch 25/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5413 - accuracy: 0.7348
Epoch 26/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5413 - accuracy: 0.7346
Epoch 27/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5408 - accuracy: 0.7341
Epoch 28/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5408 - accuracy: 0.7353
Epoch 29/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5411 - accuracy: 0.7335
Epoch 30/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5402 - accuracy: 0.7347
Epoch 31/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5406 - accuracy: 0.7347
Epoch 32/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5402 - accuracy: 0.7337
Ep

804/804 [==============================] - 1s 1ms/step - loss: 0.5384 - accuracy: 0.7374
Epoch 46/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5382 - accuracy: 0.7371
Epoch 47/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5384 - accuracy: 0.7376
Epoch 48/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5378 - accuracy: 0.7379
Epoch 49/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5383 - accuracy: 0.7368
Epoch 50/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5379 - accuracy: 0.7364
268/268 - 0s - loss: 0.5517 - accuracy: 0.7244
Loss: 0.5517321228981018, Accuracy: 0.7244315147399902
Model: "sequential_177"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_521 (Dense)            (None, 59)                3009      
_________________________________________________________________
dense_522 (Dense

In [120]:
results.sort()

In [121]:
results

[[0.7189504504203796, 50, 26],
 [0.7237317562103271, 51, 25],
 [0.7244315147399902, 59, 28],
 [0.7245481014251709, 54, 27],
 [0.7255976796150208, 55, 27],
 [0.7255976796150208, 58, 27],
 [0.7258309125900269, 50, 29],
 [0.7268804907798767, 52, 28],
 [0.7269970774650574, 57, 26],
 [0.7269970774650574, 57, 27],
 [0.7273469567298889, 52, 25],
 [0.7273469567298889, 53, 28],
 [0.7274635434150696, 56, 25],
 [0.727580189704895, 50, 27],
 [0.7276967763900757, 51, 29],
 [0.7279300093650818, 52, 29],
 [0.7281632423400879, 54, 26],
 [0.728396475315094, 55, 26],
 [0.7285131216049194, 53, 26],
 [0.7285131216049194, 59, 29],
 [0.7286297082901001, 50, 25],
 [0.7286297082901001, 56, 29],
 [0.7287463545799255, 53, 27],
 [0.7287463545799255, 57, 29],
 [0.7289795875549316, 54, 28],
 [0.7292128205299377, 56, 28],
 [0.7294460535049438, 53, 25],
 [0.729912519454956, 50, 28],
 [0.729912519454956, 51, 26],
 [0.729912519454956, 51, 27],
 [0.7302623987197876, 58, 29],
 [0.7304956316947937, 58, 28],
 [0.730845451

### Saving model weights from best result:
### first hidden layer with 6 neurons, second with 3, both with relu activation

In [122]:
final_model =  make_run_evaluate_nn_model_with_callbacks(X_train_scaled, X_test_scaled, y_train, y_test, 53, 
                               'elu', 'elu','yes', 29, 50)


Model: "sequential_178"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_524 (Dense)            (None, 53)                2703      
_________________________________________________________________
dense_525 (Dense)            (None, 29)                1566      
_________________________________________________________________
dense_526 (Dense)            (None, 1)                 30        
Total params: 4,299
Trainable params: 4,299
Non-trainable params: 0
_________________________________________________________________
Epoch 1/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5787 - accuracy: 0.7172

Epoch 00001: saving model to chcekpoints\weights.01
Epoch 2/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5598 - accuracy: 0.7263

Epoch 00002: saving model to chcekpoints\weights.02
Epoch 3/50
804/804 [==============================] - 1s 1ms/step - loss: 

804/804 [==============================] - 1s 1ms/step - loss: 0.5381 - accuracy: 0.7370

Epoch 00049: saving model to chcekpoints\weights.49
Epoch 50/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5381 - accuracy: 0.7371

Epoch 00050: saving model to chcekpoints\weights.50
268/268 - 0s - loss: 0.5511 - accuracy: 0.7313
Loss: 0.5510568618774414, Accuracy: 0.7313119769096375
